| | bag of words + logreg | tf-idf + logreg | bert | some neural network that is not transformer |
|-|-----------------------|-----------------|------|---------------------------------------------|
| accuracy | 0.7415 | 0.7455 | 0.8085 | ?|


# достать датасет

In [11]:
from nltk import WordPunctTokenizer

In [1]:
import pandas as pd

In [7]:
data = pd.read_csv("Doyle_Christie_dataset/train.csv")
data = data.drop(columns=['Unnamed: 0',])

In [31]:
test_data = pd.read_csv("Doyle_Christie_dataset/test.csv")
test_data = test_data.drop(columns=['Unnamed: 0',])
test_data['labels'] = test_data['author'].apply(lambda x: 0 if x == 'Doyle' else 1)

In [9]:
data['labels'] = data['author'].apply(lambda x: 0 if x == 'Doyle' else 1)

In [12]:
def clean_text(text):
    '''Text Preprocessing '''
    
    # Convert words to lower case
    text = text.lower()

    # Tokenize each word
    text =  WordPunctTokenizer().tokenize(text)
    
    return text

In [13]:
data['clean'] = data['sentence'].apply(clean_text)

In [14]:
data

,sentence,author,labels,clean
0,"Six horses were yoked to it, whereas the other...",Doyle,0,"[six, horses, were, yoked, to, it, ,, whereas,..."
1,"We had said nothing to him about it, so that h...",Doyle,0,"[we, had, said, nothing, to, him, about, it, ,..."
2,"You’d have done the same, if you have any manh...",Doyle,0,"[you, ’, d, have, done, the, same, ,, if, you,..."
3,It would be to your advantage as well as mine ...,Doyle,0,"[it, would, be, to, your, advantage, as, well,..."
4,"At any rate, we may take it as a hypothesis an...",Doyle,0,"[at, any, rate, ,, we, may, take, it, as, a, h..."
...,...,...,...,...
17995,"“The police can’t advertise themselves, worse ...",Christie,1,"[“, the, police, can, ’, t, advertise, themsel..."
17996,He admits to having abandoned twenty men to t...,Christie,1,"[he, admits, to, having, abandoned, twenty, me..."
17997,“Absolutely.,Christie,1,"[“, absolutely, .]"
17998,“I asked him at once about Ralph.,Christie,1,"[“, i, asked, him, at, once, about, ralph, .]"


# классический мл

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [37]:
# https://medium.com/analytics-vidhya/applying-text-classification-using-logistic-regression-a-comparison-between-bow-and-tf-idf-1f1ed1b83640

def simple_logistic_classify(X_tr, y_tr, X_test, y_test, description, _C=1.0):
#     model = LogisticRegression(C=_C).fit(X_tr, y_tr)
    model = LogisticRegression().fit(X_tr, y_tr)
    score = model.score(X_test, y_test)
    print('Test Score with', description, 'features', score)
    return model

## bow + logreg

In [29]:
bow_vect = CountVectorizer()
bow_train = bow_vect.fit_transform(data['sentence'])
bow_test = bow_vect.transform(test_data['sentence'])

In [38]:
bow_model = simple_logistic_classify(X_tr = bow_train, 
                                     y_tr = data['labels'],
                                     X_test = bow_test, 
                                     y_test = test_data['labels'], 
                                     description='bag of words')

Test Score with bag of words features 0.7415


/home/antonauna/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## tfidf + logreg

In [52]:
tfidf_vect = TfidfVectorizer()
tfidf_train = tfidf_vect.fit_transform(data['sentence'])
tfidf_test = tfidf_vect.transform(test_data['sentence'])

In [53]:
tfidf_model = simple_logistic_classify(X_tr = tfidf_train, 
                                       y_tr = data['labels'],
                                       X_test = tfidf_test,
                                       y_test = test_data['labels'], 
                                       description='tf-idf')

Test Score with tf-idf features 0.7455


# Bert

In [66]:
from datasets import load_dataset, ClassLabel, DatasetDict
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [80]:
def adjust_labels(batch):
    batch["labels"] = [int(value) for value in batch["labels"]]
    return batch


ds = Dataset.from_dict({"sentence": data['sentence'], "labels": data['labels']})
features = ds.features.copy()
features["labels"] = ClassLabel(names=["Doyle", "Christie"])
ds = ds.map(adjust_labels, batched=True, features=features)


ds_test = Dataset.from_dict({"sentence": test_data['sentence'], "labels": test_data['labels']})
features = ds.features.copy()
features["labels"] = ClassLabel(names=["Doyle", "Christie"])
ds_test = ds_test.map(adjust_labels, batched=True, features=features)

  0%|          | 0/18 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [81]:
checkpoint = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

loading configuration file config.json from cache at /home/antonauna/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.23.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file vocab.txt from cache at /home/antonauna/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/vocab.txt
l

In [83]:
def tokenize(row):
    return tokenizer(row['sentence'], truncation=True)

In [85]:
ds = ds.map(tokenize)
ds_test = ds_test.map(tokenize)

  0%|          | 0/18000 [00:00<?, ?ex/s]

  0%|          | 0/2000 [00:00<?, ?ex/s]

In [86]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors = 'pt')
accuracy = evaluate.load("accuracy")

In [87]:
id2label = {0: "Doyle", 1: "Christie"}
label2id = {"Doyle": 0, "Christie": 1}

In [97]:
# model = AutoModelForSequenceClassification.from_pretrained(
#     checkpoint, num_labels=2, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True
# )

loading configuration file config.json from cache at /home/antonauna/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Doyle",
    "1": "Christie"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "Christie": 1,
    "Doyle": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.23.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file pytorch_model.bin from cach

In [96]:
frozen_model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels=2, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True
)

loading configuration file config.json from cache at /home/antonauna/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Doyle",
    "1": "Christie"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "Christie": 1,
    "Doyle": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.23.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file pytorch_model.bin from cach

In [98]:
# for name, par in model.base_model.named_parameters():
#     if name.startswith('bert.embeddings'):
#         par.requires_grad = False
#     elif name.startswith('bert.encoder.layer.0'):
#         par.requires_grad = False
#     elif name.startswith('bert.encoder.layer.1.'):
#         par.requires_grad = False
#     elif name.startswith('bert.encoder.layer.2'):
#         par.requires_grad = False   
#     elif name.startswith('bert.encoder.layer.3'):
#         par.requires_grad = False
#     elif name.startswith('bert.encoder.layer.4'):
#         par.requires_grad = False
#     elif name.startswith('bert.encoder.layer.5'):
#         par.requires_grad = False
#     elif name.startswith('bert.encoder.layer.6'):
#         par.requires_grad = False
#     elif name.startswith('bert.encoder.layer.7'):
#         par.requires_grad = False
#     elif name.startswith('bert.encoder.layer.8'):
#         par.requires_grad = False

In [103]:
training_args = TrainingArguments(
    output_dir="frozen_bert",
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [104]:
trainer = Trainer(
    model=frozen_model,
    args=training_args,
    train_dataset=ds,
    eval_dataset=ds_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [105]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence. If sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/antonauna/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 18000
  Num Epochs = 3
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 1
  Total optimization steps = 5400


Epoch,Training Loss,Validation Loss,Accuracy
1,0.382500,0.459509,0.808500
2,0.244000,0.710962,0.798000
3,0.153300,1.063771,0.790000


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence. If sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 10
Saving model checkpoint to frozen_bert/checkpoint-1800
Configuration saved in frozen_bert/checkpoint-1800/config.json
Model weights saved in frozen_bert/checkpoint-1800/pytorch_model.bin
tokenizer config file saved in frozen_bert/checkpoint-1800/tokenizer_config.json
Special tokens file saved in frozen_bert/checkpoint-1800/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence. If sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 

TrainOutput(global_step=5400, training_loss=0.2837327512105306, metrics={'train_runtime': 132.3107, 'train_samples_per_second': 408.13, 'train_steps_per_second': 40.813, 'total_flos': 1130843093716800.0, 'train_loss': 0.2837327512105306, 'epoch': 3.0})